HAN TRAN ---
 2001708

# Question 1:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mlxtend.preprocessing
import mlxtend.frequent_patterns
import networkx as nx
import pyvis
import networkx.algorithms.community as nx_comm

In [ ]:
grocery_data = pd.read_csv('grocery_assignment.csv')
grocery_data

In [ ]:
#check the type of dataset
grocery_data.info()

In [ ]:
# TRansform the dataset into transactional format
#First, I will create a list of item description list base on the transaction_id
transaction = grocery_data.groupby(['transaction_id'])['itemDescription'].apply(list).values.tolist()
print(transaction[:2])

In [ ]:
# OR
transaction = grocery_data.groupby(['transaction_id'])['itemDescription'].apply(list).values.tolist()
transaction

In [ ]:
# Import the transaction encoder function from mlxtend
from mlxtend.preprocessing import TransactionEncoder

# Instantiate transaction encoder and identify unique items
encoder = TransactionEncoder().fit (transaction)

# One-hot encode transactions
encoded_data = encoder.transform (transaction)

# Convert one-hot encoded data to DataFrame
grocery_trans = pd.DataFrame (encoded_data, columns = encoder.columns_)

# Print the one-hot encoded transaction dataset
print(grocery_trans.head())

In [ ]:
# Check size of data
grocery_trans.shape

Extract frequent item-sets and create association rules 

In [ ]:
# The most frequent itemset 
grocery_trans.sum().sort_values(ascending = False)

In [ ]:
# I try the max_len=3, min_support=0.002
frequent_grocery_sets = mlxtend.frequent_patterns.apriori(grocery_trans, min_support = 0.002, max_len = 3, use_colnames = True)

In [ ]:
frequent_grocery_sets['item_len'] = frequent_grocery_sets['itemsets'].apply(len)
frequent_grocery_sets[frequent_grocery_sets['item_len'] == 3]

In [ ]:
print(frequent_grocery_sets.head())
len(frequent_grocery_sets)
# with this min_support 0.002 and max len=3, we have 330 itemsets

In [ ]:
#With max_len=4, min_support=0.01
frequent_grocery_set_1=mlxtend.frequent_patterns.apriori(grocery_trans, max_len=4, min_support=0.01, use_colnames=True)
print(frequent_grocery_set_1)
len(frequent_grocery_set_1)
#with this, we have itemsets in return


In [ ]:
#With max_len=4, min_support=0.02
frequent_grocery_set_2=mlxtend.frequent_patterns.apriori(grocery_trans, max_len=4, min_support=0.02, use_colnames=True)
print(frequent_grocery_set_2)
len(frequent_grocery_set_2)
#with this, we have itemsets in return

In [ ]:
#With max_len=4, min_support=0.005
frequent_grocery_set_3=mlxtend.frequent_patterns.apriori(grocery_trans, max_len=4, min_support=0.005, use_colnames=True)
print(frequent_grocery_set_3)
len(frequent_grocery_set_3)
#with this, we have itemsets in return

In [ ]:
#With max_len=3, min_support=0.05
frequent_grocery_set_4=mlxtend.frequent_patterns.apriori(grocery_trans, max_len=4, min_support=0.005, use_colnames=True)
print(frequent_grocery_set_4)
len(frequent_grocery_set_4)
#with this, we have itemsets in return

In [ ]:
#create association rules as it was done 
# Begin with the threshold=0.09
rule_1=mlxtend.frequent_patterns.association_rules(frequent_grocery_set_3, metric='confidence', min_threshold=0.09)
rule_1

In [ ]:
# With min_threshold=0.02
rule_2= mlxtend.frequent_patterns.association_rules(frequent_grocery_set_3, metric='confidence', min_threshold=0.002)
rule_2
#This threshold seems to provide a more reasonable amount of dataset 

In [ ]:
rule_2[['antecedents', 'consequents']]

Do the extracted rules

In [ ]:
 #what would you recommend to a person (i.e. what is the consequent in association rules terminology) when you now that in the basket there is already (i) sausage, (
Recommendation_1 = rule_2['antecedents'].apply(lambda x: 'sausage' in x)
rule_2[Recommendation_1]

In [ ]:
#Or we can filter like this
Recommendation= rule_2[(rule_2['antecedents']=={'sausage'})]
Recommendation
# below (consequents) is the possible items that a customer might buy after taking the sausage

In [ ]:
#what would you recommend when you now that in the basket there is already(ii) tropical fruit,
Fruit_Recommendation= rule_2['antecedents'].apply(lambda x: 'tropical fruit' in x)
rule_2[Fruit_Recommendation]
# below (consequents) is the possible items that a customer might buy after taking the tropical fruit

In [ ]:
#what would you recommend to a person when there is (iii) sausage AND tropical fruit?
Recommendation_Sausage_Fruit= rule_2['antecedents'].apply(lambda x: 'sausage ' in x and 'tropical fruit' in x)
rule_2[Recommendation_Sausage_Fruit]

In [ ]:
# As there is no assiocation rules for those who bought sausage and tropical fruit at the same time, I check the unique value for rule 2
# And we can see from the array, there's no records for those who buy sausage and tropical fruit at the same time.
#therefore, there's no recommendation in this case
rule_2.antecedents.unique()

# QUESTION 2

In [3]:
# Start with the important libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Importing the new library for network analysis
import networkx as nx

In [4]:
characters = pd.read_csv('characters.csv', encoding = 'latin')
characters.name.isnull().values.any()
characters

,id,name,bio,Gender,Job,House,Wand,Patronus,Species,Blood status,Hair colour,Eye colour,Loyalty,Skills,Birth,Death
0,0,Regulus Arcturus Black,Brother of Sirius. Used to be a Death Eater bu...,Male,NaN,Slytherin,Unknown,Non-corporeal,Human,Pure-blood,Black,NaN,NaN,Seeker,1961,1979
1,1,Sirius Black,Best friend of James Potter and godfather of H...,Male,NaN,Gryffindor,Unknown,Unknown,Human,Pure-blood,Black,Grey,Order of the Phoenix,Animagus| nonverbal magic| Charms,"3 November, 1959","18 June, 1996"
2,2,Lavender Brown,Killed by a werewolf. She was a gryffindor stu...,Female,Student,Gryffindor,Unknown,Unknown,Human,Pure-blood,Blond,Blue,Dumbledore's Army |Hogwarts School of Witchcra...,Divination,1 September 1979- 31 August 1980,"2 May, 1998"
3,3,Cho Chang,Ravenclaw student who dated Cedric Diggory and...,Female,Student,Ravenclaw,Unknown,Swan,Human,Pure-blood or half-blood,Black,Dark,Dumbledore's Army |Hogwarts School of Witchcra...,Seeker,1 September 1978-31 August 1979,NaN
4,4,Vincent Crabbe Sr.,Father of Crabbe and death-eater who escaped A...,Male,NaN,Slytherin,Unknown,Unknown,Human,Pure-blood,Dark,Dark,Lord Voldemort | Death Eaters,NaN,Pre 1964,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,60,Fluffy,Three-headed dog belonging to Rubeus Hagrid.,Male,NaN,NaN,NaN,NaN,Animal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,61,Hedwig,Harry's owl.,Female,NaN,NaN,NaN,NaN,Animal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,62,Moaning Myrtle,Ghost at Hogwards.,Female,Student,Ravenclaw,Unknown,Unknown,Ghost,Muggle-born,NaN,NaN,NaN,NaN,14 June 1928 -13 June 1929,"13 June, 1943"
63,63,Aragog,Acromantula belonging to Rubeus Hagrid.,Male,NaN,NaN,NaN,NaN,Animal,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
relations = pd.read_csv('relations.csv')
relations

,source,target
0,0,1
1,0,25
2,0,45
3,1,0
4,1,11
...,...,...
508,63,58
509,64,21
510,64,22
511,64,39


In [7]:
# I merge these two dataframe into one
# First, I change the 'source' column into 'id' column
relations= relations.rename(columns={'source':'id'})
relations.head()

,id,target
0,0,1
1,0,25
2,0,45
3,1,0
4,1,11


In [ ]:
combi = pd.merge(relations, characters, left_on="id", right_on="id")
combi

In [ ]:
#Becase the dataset relation has a source with multiple  target, the duplicates will not be removed in this situation
# extracted the most related columns in the dataframe
harry_potter=combi[[ 'id','name', 'bio', 'House', 'target']]
harry_potter

In [ ]:
# We can convert the dataframe into a network with weight and book information stored
combi_net = nx.from_pandas_edgelist(combi, 'id', 'target', ['name', 'bio'])
combi_net.edges(data = True)

In [ ]:
# Visualizing with pyvis
combi_vis = pyvis.network.Network()
combi_vis.from_nx(combi_net)
combi_vis.show('combi_vis.html')

In [ ]:
list(combi_net.nodes())

In [ ]:
# Centrality
deg = nx.degree_centrality(combi_net)
closeness = nx.closeness_centrality(combi_net)
betweenness = nx.betweenness_centrality(combi_net)
eigenvector = nx.eigenvector_centrality(combi_net)
pagerank = nx.pagerank(combi_net)

centrality_combi_net = pd.DataFrame(index = combi_net.nodes())
centrality_combi_net['degree_centrality'] = deg.values()
centrality_combi_net['closeness_centrality'] = closeness.values()
centrality_combi_net['betweenness_centrality'] = betweenness.values()
centrality_combi_net['eigenvector_centrality'] = eigenvector.values()
centrality_combi_net['pagerank'] = pagerank.values()

centrality_combi_net

In [ ]:
# Top 6 based on each centrality
for col in centrality_combi_net.columns:
    result = list(centrality_combi_net.sort_values(by = col, ascending = False).index[:6])
    print('The top 6 characters based on', col, 'in the Harry Potter movie are', result)

In [ ]:
# Correlation
centrality_combi_net.corr()

In [ ]:
#Which methods are the most similar to each other in terms of the most central characters and in terms of correlation?

print(' According to the result, the Degree centrality and pagerank are the most similar to each other')
print('Besides, the Closeness centrality and Pagerank is also similar to each other')

# Question 3:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pyvis



In [ ]:
#reload the dataset with another name
Harry_community_Combi = pd.merge(characters, relations, left_on="id", right_on="id")
Harry_community_Combi

In [ ]:
#Beacuse id is the interger type, I will change it to the string
Harry_community_Combi['id']=Harry_community_Combi['id'].astype('str')
Harry_community_Combi.info()

In [ ]:
# As we can see, from the columns Job, there are some NAN values
Harry_community_Combi['Job'].unique()


In [ ]:
# Similarly, I will check the null values in column House
Harry_community_Combi['House'].unique()
Harry_community_Combi['House'].isnull()

In [ ]:
# With the 'Unknown' value because one nodes may have multiple targets
Harry_community_Combi=Harry_community_Combi.fillna('Unknown')
Harry_community_Combi

In [ ]:
# Turn the harry_community into pandas edgelist
Harry_community_net=nx.from_pandas_edgelist(Harry_community_Combi, 'id', 'target', ['name', 'bio', 'Job'])
#checking the edges
Harry_community_net.edges(data=True)

In [ ]:
len(Harry_community_net.nodes)

In [ ]:
#Running community detection
harry_comm_detection = nx_comm.girvan_newman(Harry_community_net)
list(harry_comm_detection)

In [ ]:
# I will create the run the community detection
harry_node_selection = list(Harry_community_net.nodes())[:126]
#harry_net_selection = Harry_community_net.subgraph(harry_node_selection)
#harry_net_selection_community = nx_comm.girvan_newman(harry_net_selection)
harry_net_selection_community = nx_comm.girvan_newman(Harry_community_net)
comm_list = list(harry_net_selection_community)



In [ ]:

modularity_list = []

for comm in comm_list:
    mod = nx_comm.modularity(harry_net_selection, comm)
    modularity_list.append(mod)
    print('The modularity with ', len(comm), 'subcommunities is', mod)

In [ ]:
#From the list, we can see with 51 communities we have the highest modularity
plt.plot(modularity_list)

Compare the resulting communities and attempt to characterize the difference between communities by focusing on the information in the characters.csv file.

In [ ]:

com_51 = comm_list[51]
memb_community = []
for node in harry_net_selection:
    for i in range(len(com_51)):
        if node in com_51[i]:
            memb_community.append(i)
print(memb_community)

In [ ]:
#Load back the dataset harry_community which includes the characters.csv
harry_character = Harry_community_Combi.head(126)
harry_character

In [ ]:
# create the membership columns
harry_character['Membership'] = community_member
harry_character[['id', 'target', 'name', 'House', 'Membership']]

In [ ]:
membership_unique= harry_potter['id'].groupby(harry_potter['membership']).unique()
membership_unique

In [ ]:
# Load back the top 6 based on page rank
centrality_combi_net.sort_values(by='page rank', ascending=False).index[:6]

# As we can see, the top 6 in page ranks belongs to nodes: 39, 45, 21, 11, 1

In [ ]:
membership_unique= harry_potter['id'].groupby(harry_potter['membership']).unique()
membership_unique
# As we can see, node 1 & 11 are the important characters that appears either alone or together in many communities/membership

In [ ]:
harry_potter[harry_potter['id']=='1']
# So Sirius Black is one of the most important character so far

In [ ]:
harry_potter[harry_potter['id']=='11']
#Similarly Albus Dumbledore is one of the most important character so far